### CE310 MINI PROJECT: Genetic Programming and Symbolic Regression

For information on on symbolic regression see: 
https://en.wikipedia.org/wiki/Symbolic_regression

In [0]:
# Install DEAP toolbox
# https://deap.readthedocs.io/en/master/index.html

# Anaconda:
# easy_install deap

# Google Colab:
!pip install deap

In [0]:
# Import relevant Python modules
import operator
import math
import random
import numpy

# Import DEAP modules
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

In [0]:
# ====================================================================
# PARAMETERS
# VPopulation size: 500 vs 2000
# Tournament Size: 2 vs 5
# repeat at least 10 times per setting
no_generations = 30
no_population = 500
no_tournaments = 2
# XO rate
p_xo = 0.7
# Mutation rate
p_m  = 0.3

# Problems/Target Functions
# x4, x6, sin2, sin3, N(0,1)
index_func = 'x6'

UseSqError = False

# ====================================================================
def gauss(mu, sigma, x):
  return 1/(sigma * math.sqrt(2 * math.pi)) * math.exp(-1/2*((x-mu)/sigma)**2)

def Problem(x):
    switcher = {
        'x4': lambda x: x**4 + x**3 + x**2 + x,
        'x6': lambda x: (x**6)-2*(x**4) + (x**2),
        'sin2': lambda x: math.sin(math.pi/4+2*x),
        'sin3': lambda x: math.sin(3*(x**3)-(x**2)/7),
        'N(0,1)': lambda x: gauss(mu=0, sigma=1, x=x),
    }

    func = switcher.get(index_func, lambda: "nothing")
    return func(x)

# Points used to test the problem/target function
test_points = numpy.linspace(-math.pi,math.pi, 180).tolist();
# ====================================================================



# Define new functions
def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

# create Primitive set & classes 
if "pset" not in globals():
    pset = gp.PrimitiveSet("MAIN", 1)
    pset.addPrimitive(operator.add, 2)
    pset.addPrimitive(operator.sub, 2)
    pset.addPrimitive(operator.mul, 2)
    pset.addPrimitive(protectedDiv, 2)
    pset.addPrimitive(operator.neg, 1)
    pset.addPrimitive(math.cos, 1)
    pset.addPrimitive(math.sin, 1)
    pset.addTerminal(1)
    pset.addTerminal(-1)    
    pset.addEphemeralConstant("rand101", lambda: random.randint(-1,1))
    pset.renameArguments(ARG0='x')

    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

    
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)


def evalSymbReg(individual, points):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)

    if UseSqError:
      # Absolute distance between target curve and solution
       error = (abs(func(x) - Problem(x)) for x in points)
    else:
      # squared error
       error = (abs(func(x) - Problem(x))**2 for x in points)

    return math.fsum(error),


toolbox.register("evaluate", evalSymbReg, points=test_points)
toolbox.register("select", tools.selTournament, tournsize=no_tournaments)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=64))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=64))


random.seed(318)
random.seed()

pop = toolbox.population(n=no_population)
hof = tools.HallOfFame(1)

stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
stats_size = tools.Statistics(len)
mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
mstats.register("mdn", numpy.median)
mstats.register("avg", numpy.mean)
mstats.register("std", numpy.std)
mstats.register("min", numpy.min)
mstats.register("max", numpy.max)

pop, log = algorithms.eaSimple(pop, toolbox, p_xo, p_m, no_generations, stats=mstats,
                                       halloffame=hof, verbose=True)


In [0]:
# Plot Fitness and Size
from matplotlib import pyplot
x = numpy.arange(0, no_generations+1)
s = log.chapters['size'].select("mdn")
f = log.chapters['fitness'].select("mdn")

fig, ax = pyplot.subplots()
ax.plot(x, f/max(f), 'k--', label='Fitness')
ax.plot(x, s/max(s), 'k:', label='Size')
ax.set_xlabel('Generations')
ax.set_ylabel('Normalised Fitness/Size')
ax.set_title('Median')

#legend = ax.legend(loc='upper center', shadow=True, fontsize='x-large')
legend = ax.legend(shadow=True, fontsize='x-large')

print('Fitnes: [' + str(min(f))+', '+str(max(f))+']')
print('Size: [' + str(min(s))+', '+str(max(s))+']')
print('Evaluations: ' +str(sum(log.select("nevals"))))

pyplot.show()

In [0]:
# Best individual 
print(hof[0])

In [0]:
x = test_points
f = toolbox.compile(expr=hof[0])
#y = (func(i) for i in x)
y = numpy.empty(len(x))
for i in range(len(x)): y[i] = f(x[i])

Y = numpy.empty(len(x))
for i in range(len(x)): Y[i] = Problem(x[i])
    
from matplotlib import pyplot
fig, ax = pyplot.subplots()
ax.plot(x, y, 'r-', label='Best Solution')
ax.plot(x, Y, 'k-', label='Target func')
#legend = ax.legend(loc='upper center', shadow=True, fontsize='x-large')
legend = ax.legend(shadow=True, fontsize='x-large')

pyplot.show()